In [ ]:
import os

import numpy as np
import matplotlib.pyplot as pl
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
import pandas as pd
from scipy.optimize import curve_fit
import statsmodels.api as sm
import statsmodels.formula.api as smf
import xarray as xr

In [ ]:
os.makedirs('../plots', exist_ok=True)

In [ ]:
pl.style.use('../defaults.mplstyle')

In [ ]:
noch4r = xr.open_dataset('../results/noCH4R.nc')

In [ ]:
ch4r = xr.open_dataset('../results/CH4R.nc')

In [ ]:
noch4r_lifetime = xr.open_dataset('../results/noCH4R_lifetime.nc')

In [ ]:
noch4r.temperature

In [ ]:
ch4r.temperature

In [ ]:
ecs_tcr = pd.read_csv('../results/ecs_tcr.csv', index_col=0)

In [ ]:
temp_norm_20032022 = np.ones(21)
temp_norm_20032022[0] = 0.5
temp_norm_20032022[-1] = 0.5

In [ ]:
colors = {
    'ssp119': '#00a9cf',
    'COFFEE1.1 EN_NPi2020_400f_lowBECCS': '#B8BDAA',
    'ssp534-over': '#92397a'
}

In [ ]:
scenarios = ['ssp119', 'COFFEE1.1 EN_NPi2020_400f_lowBECCS', 'ssp534-over']
labels = {
    'ssp119': 'SSP1-1.9',
    'COFFEE1.1 EN_NPi2020_400f_lowBECCS': 'IMP-Neg (C2)',
    'ssp534-over': 'SSP5-3.4-OS'
}

In [ ]:
def powerfit(x, a, b, c):
    return a + b * x**c

# def powerfit(x, b):
#     return 1.5 + b * x**0.5

In [ ]:
quantiles = [.05, .25, .50, .75, .95]
mod = {}
res_all = {}
x_p = {}
df_p = {}
p = {}
cov = {}
exceeds = {}

for iscen, scenario in enumerate(scenarios):
    over1p5 = (
        np.max(
            noch4r.temperature.loc[
                dict(scenario=scenario, layer=0)
            ] - np.average(
                noch4r.temperature.loc[
                    dict(scenario=scenario, layer=0, timebound=np.arange(2003,2024))
                ], axis=0, weights=temp_norm_20032022
            ) + 1.03, axis=0
        )
    ) >= 1.5
    x = (
        -np.sum(
            ch4r.emissions.loc[
                dict(scenario=scenario)
            ] - noch4r.emissions.loc[
                dict(scenario=scenario, specie='CH4')
            ], axis=0
        ).data[over1p5]/1000
    )
    y = (
        np.max(
            noch4r.temperature.loc[
                dict(scenario=scenario, layer=0)
            ] - np.average(
                noch4r.temperature.loc[
                    dict(scenario=scenario, layer=0, timebound=np.arange(2003,2024))
                ], axis=0, weights=temp_norm_20032022
            ) + 1.03, axis=0
        ).data[over1p5]
    )
    exceeds[scenario] = (
        np.max(
            ch4r.temperature.loc[
                dict(scenario=scenario)
            ] - np.average(
                ch4r.temperature.loc[
                    dict(scenario=scenario, timebound=np.arange(2003,2024))
                ], axis=0, weights=temp_norm_20032022
            ) + 1.03, axis=0
        ).data > 1.5
    )
    xe = (
        -np.sum(
            ch4r.emissions.loc[
                dict(scenario=scenario)
            ] - noch4r.emissions.loc[
                dict(scenario=scenario, specie='CH4')
            ], axis=0
        ).data[over1p5 & ~exceeds[scenario]]/1000
    )
    ye = (
        np.max(
            noch4r.temperature.loc[
                dict(scenario=scenario, layer=0)
            ] - np.average(
                noch4r.temperature.loc[
                    dict(scenario=scenario, layer=0, timebound=np.arange(2003,2024))
                ], axis=0, weights=temp_norm_20032022
            ) + 1.03, axis=0
        ).data[over1p5 & ~exceeds[scenario]]
    )
    
    d = {'peak': ye, 'ch4': xe}
    #d = {'peak': ye-1.5, 'ch4': xe}
    df = pd.DataFrame(data=d)
    
    # scipy curve_fit
    p[scenario], cov[scenario] = curve_fit(powerfit, xe, ye)
    
    # use the best estimate of the exponent from this to feed into the quantile regression
    
    # statsmodels fit
    mod[scenario] = smf.quantreg(f'peak ~  I(ch4 ** {p[scenario][2]})', df)
    #mod[scenario] = smf.quantreg(f'peak ~ I(ch4 ** 0.5) - 1', df)
    
    # get all result instances in a list
    res_all[scenario] = [mod[scenario].fit(q=q) for q in quantiles]
    x_p[scenario] = np.linspace(0, x.max(), 100)
    df_p[scenario] = pd.DataFrame({'ch4': x_p})

In [ ]:
fig, ax = pl.subplots(1, 3, figsize=(18/2.54, 6/2.54))
for iscen, scenario in enumerate(scenarios):
    ax[iscen].scatter(
        -np.sum(
            ch4r.emissions.loc[
                dict(scenario=scenario)
            ] - noch4r.emissions.loc[
                dict(scenario=scenario, specie='CH4')
            ], axis=0
        )[~exceeds[scenario]]/1000, 
        np.max(
            noch4r.temperature.loc[
                dict(scenario=scenario, layer=0)
            ] - np.average(
                noch4r.temperature.loc[
                    dict(scenario=scenario, layer=0, timebound=np.arange(2003,2024))
                ], axis=0, weights=temp_norm_20032022
            ) + 1.03, axis=0
        )[~exceeds[scenario]],
        label=labels[scenario],
        s=8,
        lw=0,
        alpha=0.8,
        color=colors[scenario],
        marker="o"
    )
    ax[iscen].scatter(
        -np.sum(
            ch4r.emissions.loc[
                dict(scenario=scenario)
            ] - noch4r.emissions.loc[
                dict(scenario=scenario, specie='CH4')
            ], axis=0
        )[exceeds[scenario]]/1000, 
        np.max(
            noch4r.temperature.loc[
                dict(scenario=scenario, layer=0)
            ] - np.average(
                noch4r.temperature.loc[
                    dict(scenario=scenario, layer=0, timebound=np.arange(2003,2024))
                ], axis=0, weights=temp_norm_20032022
            ) + 1.03, axis=0
        )[exceeds[scenario]],
        label=labels[scenario],
        s=8,
        lw=0.4,
        alpha=0.8,
        edgecolor=colors[scenario],
        color="None",
        marker="o"
    )
#     for qm, res in zip(quantiles, res_all[scenario]):
#         q[qm] = res.predict({'peak': x_p[scenario]})
        # get prediction for the model and plot
        # here we use a dict which works the same way as the df in ols
#     ax[iscen].fill_between(x_p[scenario], q[.05], q[.95], color='k', alpha=0.3, lw=0)
#     ax[iscen].plot(x_p[scenario], q[.5], color='k')

    ax[iscen].set_title(f'({chr(97+iscen)}) {labels[scenario]}')
    ax[iscen].set_xlabel('Cumulative methane removal (GtCH$_4$)')
    ax[iscen].set_ylabel('Peak warming (K)')
    ax[iscen].set_xlim(-5, 130)
    ax[iscen].set_ylim(1, 4.5)
fig.tight_layout()
pl.savefig('../plots/cumCH4_peakT.png')
pl.savefig('../plots/cumCH4_peakT.pdf')

In [ ]:
fig, ax = pl.subplots(1, 3, figsize=(18/2.54, 6/2.54))
for iscen, scenario in enumerate(scenarios):
    ax[iscen].scatter(
        -np.sum(
            ch4r.emissions.loc[
                dict(scenario=scenario)
            ] - noch4r.emissions.loc[
                dict(scenario=scenario, specie='CH4')
            ], axis=0
        )[~exceeds[scenario]]/1000, 
        np.max(
            noch4r.temperature.loc[
                dict(scenario=scenario, layer=0)
            ] - np.average(
                noch4r.temperature.loc[
                    dict(scenario=scenario, layer=0, timebound=np.arange(2003,2024))
                ], axis=0, weights=temp_norm_20032022
            ) + 1.03, axis=0
        )[~exceeds[scenario]],
        label=labels[scenario],
        s=8,
        lw=0,
        alpha=0.8,
        color=colors[scenario],
        marker="o"
    )
    ax[iscen].scatter(
        -np.sum(
            ch4r.emissions.loc[
                dict(scenario=scenario)
            ] - noch4r.emissions.loc[
                dict(scenario=scenario, specie='CH4')
            ], axis=0
        )[exceeds[scenario]]/1000, 
        np.max(
            noch4r.temperature.loc[
                dict(scenario=scenario, layer=0)
            ] - np.average(
                noch4r.temperature.loc[
                    dict(scenario=scenario, layer=0, timebound=np.arange(2003,2024))
                ], axis=0, weights=temp_norm_20032022
            ) + 1.03, axis=0
        )[exceeds[scenario]],
        label=labels[scenario],
        s=8,
        lw=0.4,
        alpha=0.8,
        edgecolor=colors[scenario],
        color="None",
        marker="o"
    )
    
    q = {}
    for qm, res in zip(quantiles, res_all[scenario]):
        q[qm] = res.predict({'ch4': x_p[scenario]})
        # get prediction for the model and plot
        # here we use a dict which works the same way as the df in ols
    #ax[iscen].fill_between(x_p[scenario], q[.05], q[.95], color='k', alpha=0.3, lw=0)
    #ax[iscen].plot(x_p[scenario], q[.5], color='k')
    ax[iscen].fill_between(x_p[scenario], q[.05], q[.95], color='k', alpha=0.3, lw=0)
    ax[iscen].plot(x_p[scenario], q[.5], color='k')

    ax[iscen].set_title(f'({chr(97+iscen)}) {labels[scenario]}')
    ax[iscen].set_xlabel('Cumulative methane removal (GtCH$_4$)')
    ax[iscen].set_ylabel('Peak warming (°C)')
    ax[iscen].set_xlim(0, 120)
    ax[iscen].set_ylim(1, 4.5)
    

# Custom legend
legend_elements = [
    Line2D([0], [0], marker='o', color="None", label='Limited to 1.5°C', markerfacecolor='k', markersize=np.sqrt(8)),
    Line2D([0], [0], marker='o', color='None', label='Not Limited to 1.5°C', markerfacecolor='w', markeredgecolor='k', markersize=np.sqrt(8), lw=0.4),
    Patch(facecolor='0.7', lw=0, label='Quantile regression 5-95%'),
    Line2D([0], [0], color='k', lw=1.5, label='Quantile regression median'),
]
ax[0].legend(handles=legend_elements, loc='upper left', frameon=False)

fig.tight_layout()
pl.savefig('../plots/cumCH4_peakT_quant.png')
pl.savefig('../plots/cumCH4_peakT_quant.pdf')

In [ ]:
for scenario in scenarios:
    for qidx in [0, 2, 4]:
        print(scenario, quantiles[qidx])
        print(res_all[scenario][qidx].params)
        print()

In [ ]:
fig, ax = pl.subplots(figsize=(9/2.54, 9/2.54))
for scenario in scenarios:
    ax.plot(
        noch4r.timepoint, 
        np.median(noch4r.emissions.loc[dict(scenario=scenario, specie='CH4')], axis=1), color=colors[scenario],
        ls = '--'
    );
    ax.plot(
        ch4r.timepoint, 
        np.median(ch4r.emissions.loc[dict(scenario=scenario)], axis=1), color=colors[scenario],
        label=labels[scenario]
    );
    
hands, labs = ax.get_legend_handles_labels()

hands.append(Line2D([0], [0], color='k', lw=1.5))
hands.append(Line2D([0], [0], color='k', ls='--', lw=1.5))
labs.append('With methane removal')
labs.append('Unaltered scenario')

ax.set_xlim(2015, 2150)
ax.set_ylim(-400, 600)
ax.axhline(0, lw=0.5, ls=':', color='k')
ax.set_ylabel('Mt CH$_4$ yr$^{-1}$')
ax.legend(hands, labs, frameon=False)
ax.set_title('Ensemble median net methane emissions');
pl.savefig('../plots/netCH4emissions.png')
pl.savefig('../plots/netCH4emissions.pdf')

In [ ]:
fig, ax = pl.subplots(figsize=(9/2.54, 9/2.54))
for scenario in scenarios:
    ax.plot(
        ch4r.timepoint, 
        np.median(noch4r.emissions.loc[dict(scenario=scenario, specie='CH4')] - ch4r.emissions.loc[dict(scenario=scenario)], axis=1), color=colors[scenario],
        label=labels[scenario]
    );
ax.set_xlim(2015, 2150)
ax.set_ylim(0, 900)
ax.axhline(0, lw=0.5, ls=':', color='k')
ax.set_ylabel('Mt CH$_4$ yr$^{-1}$')
ax.legend(frameon=False)
ax.set_title('Ensemble median gross methane removal');
pl.savefig('../plots/grossCH4removal.png')
pl.savefig('../plots/grossCH4removal.pdf')

In [ ]:
fig, ax = pl.subplots(1, 2, figsize=(18/2.54, 9/2.54))
for scenario in scenarios:
    ax[1].plot(
        noch4r.timepoint, 
        np.median(noch4r.emissions.loc[dict(scenario=scenario, specie='CH4')], axis=1), color=colors[scenario],
        ls='--',
    );
    ax[1].plot(
        ch4r.timepoint, 
        np.median(ch4r.emissions.loc[dict(scenario=scenario)], axis=1), color=colors[scenario],
        label=labels[scenario]
    );
ax[1].set_xlim(2000, 2300)
ax[1].set_ylim(-400, 600)
ax[1].axhline(0, lw=0.5, ls=':', color='k')
ax[1].set_ylabel('Mt CH$_4$ yr$^{-1}$')
ax[1].set_title('(b) Ensemble median net methane emissions');

hands, labs = ax[1].get_legend_handles_labels()
hands.append(Line2D([0], [0], color='k', lw=1.5))
hands.append(Line2D([0], [0], color='k', ls='--', lw=1.5))
labs.append('With methane removal')
labs.append('Unaltered scenario')
ax[1].legend(hands, labs, frameon=False)

for scenario in scenarios:
    ax[0].plot(
        ch4r.timepoint, 
        np.median(noch4r.emissions.loc[dict(scenario=scenario, specie='CH4')] - ch4r.emissions.loc[dict(scenario=scenario)], axis=1), color=colors[scenario],
        label=labels[scenario]
    );
ax[0].set_xlim(2000, 2300)
ax[0].set_ylim(0, 900)
ax[0].axhline(0, lw=0.5, ls=':', color='k')
ax[0].set_ylabel('Mt CH$_4$ yr$^{-1}$')
ax[0].legend()
ax[0].set_title('(a) Ensemble median gross methane removal');
fig.tight_layout()

pl.savefig('../plots/netCH4emissions_grossCH4removal.png')
pl.savefig('../plots/netCH4emissions_grossCH4removal.pdf')

In [ ]:
fig, ax = pl.subplots(figsize=(9/2.54, 9/2.54))
for scenario in scenarios:
    ax.plot(
        noch4r.timebound, 
        np.median(noch4r.temperature.loc[dict(scenario=scenario, layer=0)], axis=1), color=colors[scenario],
        ls='--',
    );
    ax.plot(
        ch4r.timebound, 
        np.median(ch4r.temperature.loc[dict(scenario=scenario)], axis=1), color=colors[scenario],
        label=labels[scenario]
    );
hands, labs = ax.get_legend_handles_labels()
hands.append(Line2D([0], [0], color='k', lw=1.5))
hands.append(Line2D([0], [0], color='k', ls='--', lw=1.5))
labs.append('With methane removal')
labs.append('Unaltered scenario')
ax.set_xlim(2000, 2300)
ax.set_ylim(0.7, 2.5)
ax.axhline(0, lw=0.5, ls=':', color='k')
ax.set_ylabel('°C relative to 1850-1900')
ax.axhline(1.5, lw=0.5, ls=':', color='k')
ax.legend(hands, labs, frameon=False)
ax.set_title('Ensemble median global mean surface temperature');
pl.savefig('../plots/temperature.png')
pl.savefig('../plots/temperature.pdf')

In [ ]:
fig, ax = pl.subplots(figsize=(9/2.54, 9/2.54))
for scenario in scenarios:
    ax.plot(
        noch4r.timebound, 
        np.median(
            noch4r.temperature.loc[dict(scenario=scenario, layer=0)] - ch4r.temperature.loc[dict(scenario=scenario)],
            axis=1
        ),
        color=colors[scenario],
        label=labels[scenario]
    );

ax.set_xlim(2000, 2300)
ax.set_ylim(0, 0.8)
ax.set_ylabel('°C')
ax.axhline(1.5, lw=0.5, ls=':', color='k')
ax.legend(frameon=False)
ax.set_title('Ensemble median avoided warming');
pl.savefig('../plots/avoidedwarming.png')
pl.savefig('../plots/avoidedwarming.pdf')

In [ ]:
fig, ax = pl.subplots(figsize=(9/2.54, 9/2.54))
for scenario in scenarios:
    ax.plot(
        noch4r.timebound, 
        np.median(noch4r.concentration.loc[dict(scenario=scenario, specie='CH4')], axis=1), color=colors[scenario],
        ls='--'
    );
    ax.plot(
        ch4r.timebound, 
        np.median(ch4r.concentration.loc[dict(scenario=scenario)], axis=1), color=colors[scenario],
        label=labels[scenario]
    );
hands, labs = ax.get_legend_handles_labels()
hands.append(Line2D([0], [0], color='k', lw=1.5))
hands.append(Line2D([0], [0], color='k', ls='--', lw=1.5))
labs.append('With methane removal')
labs.append('Unaltered scenario')
ax.set_xlim(2000, 2300)
ax.set_ylim(0, 2300)
ax.set_ylabel('ppb')
ax.legend(hands, labs, frameon=False)
ax.set_title('Ensemble median methane concentrations');
pl.savefig('../plots/CH4concentrations.png')
pl.savefig('../plots/CH4concentrations.pdf')

In [ ]:
fig, ax = pl.subplots(figsize=(9/2.54, 9/2.54))
for scenario in scenarios:
    ax.plot(
        noch4r.timebound, 
        np.median(noch4r.forcing.loc[dict(scenario=scenario, specie='CH4')], axis=1), color=colors[scenario],
        ls='--',
    );
    ax.plot(
        ch4r.timebound, 
        np.median(ch4r.forcing_ch4.loc[dict(scenario=scenario)], axis=1), color=colors[scenario],
        label=labels[scenario]
    );
hands, labs = ax.get_legend_handles_labels()
hands.append(Line2D([0], [0], color='k', lw=1.5))
hands.append(Line2D([0], [0], color='k', ls='--', lw=1.5))
labs.append('With methane removal')
labs.append('Unaltered scenario')
ax.set_xlim(2000, 2300)
ax.set_xlim(2000, 2300)
ax.set_ylim(-1, 0.8)
ax.axhline(0, lw=0.5, ls=':', color='k')
ax.set_ylabel('W m$^{-2}$')
ax.legend(hands, labs, frameon=False)
ax.set_title('Ensemble median methane radiative forcing');
pl.savefig('../plots/CH4forcing.png')
pl.savefig('../plots/CH4forcing.pdf')

In [ ]:
fig, ax = pl.subplots(figsize=(9/2.54, 9/2.54))
for scenario in scenarios:
    ax.plot(
        noch4r.timebound, 
        np.median(noch4r.forcing.loc[dict(scenario=scenario, specie='Ozone')], axis=1), color=colors[scenario],
        ls='--',
    );
    ax.plot(
        ch4r.timebound, 
        np.median(ch4r.forcing_o3.loc[dict(scenario=scenario)], axis=1), color=colors[scenario],
        label=labels[scenario]
    );
hands, labs = ax.get_legend_handles_labels()
hands.append(Line2D([0], [0], color='k', lw=1.5))
hands.append(Line2D([0], [0], color='k', ls='--', lw=1.5))
labs.append('With methane removal')
labs.append('Unaltered scenario')
ax.set_xlim(2000, 2300)
ax.set_ylim(-0.1, 0.6)
ax.set_ylabel('W m$^{-2}$')
ax.axhline(0, lw=0.5, ls=':', color='k')
ax.legend(hands, labs, frameon=False)
ax.set_title('Ensemble median ozone radiative forcing');
pl.savefig('../plots/O3forcing.png')
pl.savefig('../plots/O3forcing.pdf')

In [ ]:
fig, ax = pl.subplots(figsize=(9/2.54, 9/2.54))
for scenario in scenarios:
    ax.plot(
        noch4r.timebound, 
        np.median(noch4r.forcing_sum.loc[dict(scenario=scenario)], axis=1), color=colors[scenario],
        ls = '--',
    );
    ax.plot(
        ch4r.timebound, 
        np.median(ch4r.forcing_sum.loc[dict(scenario=scenario)], axis=1), color=colors[scenario],
        label=labels[scenario]
    );
hands, labs = ax.get_legend_handles_labels()
hands.append(Line2D([0], [0], color='k', lw=1.5))
hands.append(Line2D([0], [0], color='k', ls='--', lw=1.5))
labs.append('With methane removal')
labs.append('Unaltered scenario')
ax.set_xlim(2000, 2300)
ax.set_ylim(1, 5)
ax.set_ylabel('W m$^{-2}$')
ax.axhline(0, lw=0.5, ls=':', color='k')
ax.legend(hands, labs, frameon=False)
ax.set_title('Ensemble median radiative forcing');
pl.savefig('../plots/totalforcing.png')
pl.savefig('../plots/totalforcing.pdf')

In [ ]:
noch4r_lifetime.loc[dict(scenario=scenario)]

In [ ]:
fig, ax = pl.subplots(figsize=(9/2.54, 9/2.54))
for scenario in scenarios:
    ax.plot(
        noch4r.timebound, 
        np.median(noch4r_lifetime.loc[dict(scenario=scenario)].__xarray_dataarray_variable__, axis=1), color=colors[scenario],
        ls = '--'
    );
    ax.plot(
        ch4r.timebound, 
        np.median(ch4r.lifetime.loc[dict(scenario=scenario)], axis=1), color=colors[scenario],
        label=labels[scenario]
    );
hands, labs = ax.get_legend_handles_labels()
hands.append(Line2D([0], [0], color='k', lw=1.5))
hands.append(Line2D([0], [0], color='k', ls='--', lw=1.5))
labs.append('With methane removal')
labs.append('Unaltered scenario')
ax.set_xlim(2000, 2300)
ax.set_ylim(6, 11)
ax.set_ylabel('yr')
ax.axhline(0, lw=0.5, ls=':', color='k')
ax.legend(hands, labs, frameon=False)
ax.set_title('Ensemble median methane lifetime');
pl.savefig('../plots/CH4lifetime.png')
pl.savefig('../plots/CH4lifetime.pdf')

In [ ]:
noch4r.emissions.loc[dict(scenario=scenario, specie='CH4')].shape

In [ ]:
def lighten_color(color, amount=0.5):
    """
    Lightens the given color by multiplying (1-luminosity) by the given amount.
    Input can be matplotlib color string, hex string, or RGB tuple.

    Examples:
    >> lighten_color('g', 0.3)
    >> lighten_color('#F034A3', 0.6)
    >> lighten_color((.3,.55,.1), 0.5)
    """
    import matplotlib.colors as mc
    import colorsys
    try:
        c = mc.cnames[color]
    except:
        c = color
    c = colorsys.rgb_to_hls(*mc.to_rgb(c))
    return colorsys.hls_to_rgb(c[0], 1 - amount * (1 - c[1]), c[2])

In [ ]:
fig, ax = pl.subplots(1, 3, figsize=(18/2.54, 6/2.54))
for iscen, scenario in enumerate(scenarios):

    ax[iscen].fill_between(
        ch4r.timepoint, 
        np.percentile(
            (
                noch4r.emissions.loc[dict(scenario=scenario, specie='CH4')] -
                ch4r.emissions.loc[dict(scenario=scenario)]
            ),
            99,
            axis=1
        ),
        color=lighten_color(colors[scenario], 0.2),
        label='99%',
    );
    
    ax[iscen].fill_between(
        ch4r.timepoint, 
        np.percentile(
            (
                noch4r.emissions.loc[dict(scenario=scenario, specie='CH4')] -
                ch4r.emissions.loc[dict(scenario=scenario)]
            ),
            95,
            axis=1
        ),
        color=lighten_color(colors[scenario], 0.4),
        label='95%',
    );
    
    ax[iscen].fill_between(
        ch4r.timepoint, 
        np.percentile(
            (
                noch4r.emissions.loc[dict(scenario=scenario, specie='CH4')] -
                ch4r.emissions.loc[dict(scenario=scenario)]
            ),
            90,
            axis=1
        ),
        color=lighten_color(colors[scenario], 0.6),
        label='90%',
    );
    
    ax[iscen].fill_between(
        ch4r.timepoint, 
        np.percentile(
            (
                noch4r.emissions.loc[dict(scenario=scenario, specie='CH4')] -
                ch4r.emissions.loc[dict(scenario=scenario)]
            ),
            80,
            axis=1
        ),
        color=lighten_color(colors[scenario], 0.8),
        label='80%',
    );
    
    ax[iscen].plot(
        ch4r.timepoint, 
        np.median(noch4r.emissions.loc[dict(scenario=scenario, specie='CH4')] - ch4r.emissions.loc[dict(scenario=scenario)], axis=1), 
        color=colors[scenario],
        label='50%',
    );
    ax[iscen].set_xlim(2000, 2300)
    ax[iscen].set_ylim(0,1500)
    #ax.set_ylim(0, 900)
    ax[iscen].set_ylabel('Mt CH$_4$ yr$^{-1}$')
    ax[iscen].legend(frameon=False)
    ax[iscen].set_title(f'Gross CH$_4$ removal {labels[scenario]}');
fig.tight_layout()
pl.savefig('../plots/grossCH4removalpercentiles.png')
pl.savefig('../plots/grossCH4removalpercentiles.pdf')

In [ ]:
fig, ax = pl.subplots(1, 3, figsize=(18/2.54, 6/2.54))
for iscen, scenario in enumerate(scenarios):
    ax[iscen].scatter(
        -np.sum(
            ch4r.emissions.loc[
                dict(scenario=scenario)
            ] - noch4r.emissions.loc[
                dict(scenario=scenario, specie='CH4')
            ], axis=0
        )/1000, 
        ecs_tcr["TCR"],
        label=labels[scenario],
        s=5,
        lw=0,
        alpha=0.8,
        color=colors[scenario]
    )
    ax[iscen].set_title(f'({chr(iscen+97)}) {labels[scenario]}')
    ax[iscen].set_xlabel('Cumulative methane removal (GtCH$_4$)')
    ax[iscen].set_ylabel('Transient climate response (°C)')
    ax[iscen].set_xlim(-5, 130)
    ax[iscen].set_ylim(0.85, 3.05)
fig.tight_layout()
pl.savefig('../plots/cumCH4_TCR.png')
pl.savefig('../plots/cumCH4_TCR.pdf')

In [ ]:
fig, ax = pl.subplots(1, 3, figsize=(18/2.54, 6/2.54))
for iscen, scenario in enumerate(scenarios):
    ax[iscen].scatter(
        -np.sum(
            ch4r.emissions.loc[
                dict(scenario=scenario)
            ] - noch4r.emissions.loc[
                dict(scenario=scenario, specie='CH4')
            ], axis=0
        )/1000, 
        ecs_tcr["ECS"],
        label=labels[scenario],
        s=5,
        lw=0,
        alpha=0.8,
        color=colors[scenario]
    )
    ax[iscen].set_title(f'({chr(iscen+97)}) {labels[scenario]}')
    ax[iscen].set_xlabel('Cumulative methane removal (GtCH$_4$)')
    ax[iscen].set_ylabel('Equilibrium climate sensitivity (°C)')
    ax[iscen].set_xlim(-5, 130)
    ax[iscen].set_ylim(1, 7.5)
fig.tight_layout()
pl.savefig('../plots/cumCH4_ECS.png')
pl.savefig('../plots/cumCH4_ECS.pdf')

In [ ]:
fig, ax = pl.subplots(1, 3, figsize=(18/2.54, 6/2.54))
for iscen, scenario in enumerate(scenarios):
    ax[iscen].scatter(
        -np.sum(
            ch4r.emissions.loc[
                dict(scenario=scenario)
            ] - noch4r.emissions.loc[
                dict(scenario=scenario, specie='CH4')
            ], axis=0
        )/1000, 
        noch4r.forcing.loc[
            dict(scenario=scenario, specie='Aerosol-radiation interactions', timebound=np.arange(2005, 2015))
        ].mean(dim='timebound')+noch4r.forcing.loc[
            dict(scenario=scenario, specie='Aerosol-cloud interactions', timebound=np.arange(2005, 2015))
        ].mean(dim='timebound'),
        label=labels[scenario],
        s=5,
        lw=0,
        alpha=0.8,
        color=colors[scenario]
    )
    ax[iscen].set_title(f'({chr(iscen+97)}) {labels[scenario]}')
    ax[iscen].set_xlabel('Cumulative methane removal (GtCH$_4$)')
    ax[iscen].set_ylabel('2005-2014 aerosol forcing (W m$^{-2}$)')
    ax[iscen].set_xlim(-5, 130)
    ax[iscen].set_ylim(-2.7, 0.1)
fig.tight_layout()
pl.savefig('../plots/cumCH4_aerosolF.png')
pl.savefig('../plots/cumCH4_aerosolF.pdf')

In [ ]:
zec = pd.read_csv('../results/zec_esm-bell.csv', index_col=0)
zec

In [ ]:
fig, ax = pl.subplots(1, 3, figsize=(18/2.54, 6/2.54))
for iscen, scenario in enumerate(scenarios):
    ax[iscen].scatter(
        -np.sum(
            ch4r.emissions.loc[
                dict(scenario=scenario)
            ] - noch4r.emissions.loc[
                dict(scenario=scenario, specie='CH4')
            ], axis=0
        )/1000, 
        zec.zec50_1000,
        label=labels[scenario],
        s=5,
        lw=0,
        alpha=0.8,
        color=colors[scenario]
    )
    ax[iscen].set_xlabel('Cumulative methane removal (GtCH$_4$)')
    ax[iscen].set_ylabel('ZEC50, °C (1000 PgC bell experiment)')
    ax[iscen].set_xlim(-5, 130)
    ax[iscen].set_ylim(-0.35, 0.55)
    ax[iscen].set_title(f'({chr(iscen+97)}) {labels[scenario]}')   
fig.tight_layout()
pl.savefig('../plots/cumCH4_zec50_1000.png')
pl.savefig('../plots/cumCH4_zec50_1000.pdf')

In [ ]:
fig, ax = pl.subplots(1, 3, figsize=(18/2.54, 6/2.54))
for iscen, scenario in enumerate(scenarios):
    ax[iscen].scatter(
        -np.sum(
            ch4r.emissions.loc[
                dict(scenario=scenario)
            ] - noch4r.emissions.loc[
                dict(scenario=scenario, specie='CH4')
            ], axis=0
        )/1000, 
        zec.zec100_1000,
        label=labels[scenario],
        s=5,
        lw=0,
        alpha=0.8,
        color=colors[scenario]
    )
    ax[iscen].set_xlabel('Cumulative methane removal (GtCH$_4$)')
    ax[iscen].set_ylabel('ZEC100 °C (1000 PgC bell experiment)')
    ax[iscen].set_xlim(-5, 130)
    ax[iscen].set_ylim(-0.55, 0.95)
    ax[iscen].set_title(f'({chr(iscen+97)}) {labels[scenario]}')
fig.tight_layout()
pl.savefig('../plots/cumCH4_zec100_1000.png')
pl.savefig('../plots/cumCH4_zec100_1000.pdf')